In [22]:
import glob
import os.path as osp
import numpy as np

rootpath = "/home/radiology_datas/JHU-radiology/20170509/"
path_CN = glob.glob(rootpath + "CN/*/*.pkl")
path_AD = glob.glob(rootpath + "AD/*/*.pkl")
path_EMCL = glob.glob(rootpath + "EMCI/*/*.pkl")
path_LMCL = glob.glob(rootpath + "LMCI/*/*.pkl")
path_SMC = glob.glob(rootpath + "SMC/*/*.pkl")

label_CN = 0
label_AD = 1
label_EMCL = 2
label_LMCL = 3
label_SMC = 4

print(path_AD[0])
print(len(path_AD))

/home/radiology_datas/JHU-radiology/20170509/AD/002_S_5018/cropbrain_flipRL_ADNI_002_S_5018_MR_MT1__N3m_Br_20121206140426408_S174291_I349888_MNI.pkl
1114


In [23]:
import pickle

with open(path_CN[0], 'rb')as rf:
    image = pickle.load(rf, encoding='latin1')

#image

In [24]:
import glob
import os.path as osp
import random
import numpy as np
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader
import torchvision
from torchvision import models, transforms

In [16]:
class ImageTransform():
    def __init__(self):
        self.data_transform = transforms.Compose(
            transforms.ToTensor()
        )
    
    def __call__(self, img):
        return self.data_transform(img)

In [17]:
class BrainDataset(data.Dataset):

    def __init__(self, file_list, label, transform=None):
        self.file_list = file_list
        self.label = label
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img_paht = self.file_list[index]
        with open(file_path, 'rb') as rf:
            img = pickle.load(rf, encoding='latin1')

        img_transformed = self.transform(img)
        return img_transformed, self.label

    def __call__(self, index):
        return self.__getitem__(index)



In [21]:
dataset = BrainDataset(file_list=path_CN, label=label_CN, transform=ImageTransform)
n_train = int(len(dataset) * 0.7)
n_val = int(len(dataset) - n_train)
train_set, val_set = torch.utils.data.random_split(dataset, [n_train, n_val])
